In [63]:
# '''
# Image style transfer using convolutional neural networks.

# A style image, content image and output image exist. The thrust of the script is
# to minimize the content difference between the content image and the output image,
# whilst simulatenously minimizing the style difference between the style image
# and the output image. The output image exists as an intermediary between the
# content and style images in which the structural elements of the content image
# are blended with the stylistic components of the style image.
# '''

# # util
# import numpy as np
# import time
# import scipy
# from PIL import Image
# from os import listdir, mkdir, getcwd

# # keras backend
# from keras.applications.vgg16 import VGG16  # "OxfordNet", pretrained network
# from keras import backend as K
# K.set_image_dim_ordering('tf')

# def content_loss(content_np, composition_np):
#     '''
#     Calculate and return the content loss between the content image and the
#     combination image. The scaled Euclidean distance between feature
#     representations of the content and combination images.
#     '''
#     return K.sum(K.square(composition_np - content_np))


# def gram_matrix(image_np):
#     '''
#     Captures information about which features within an image tend to 
#     activate with one another. Captures aggregate information about a particular
#     image whilst ignoring internal, structural detail.
#     '''
#     features = K.batch_flatten(K.permute_dimensions(image_np, (2, 0, 1)))
#     gram = K.dot(features, K.transpose(features))
    
#     return gram


# def style_loss(style_np, composition_np):
#     '''
#     Calculate style loss between style reference image and combination image.
#     Calculated as the scaled Frobenius norm of the difference between the Gram
#     matrices of the style/combination images.
#     '''
#     S = gram_matrix(style_np)
#     C = gram_matrix(composition_np)
#     channels = 3
#     size = height * width
    
#     return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))


# def total_variation_loss(image_np):
#     '''
#     Reduce the noise present within the combined image by encnouraging spatial
#     smoothness via regularization.
#     '''
#     a = K.square(image_np[:, :height - 1, :width - 1, :] - image_np[:, 1:, :width - 1, :])
#     b = K.square(image_np[:, :height - 1, :width - 1, :] - image_np[:, :height - 1, 1:, :])
    
#     return K.sum(K.pow(a + b, 1.25))


# def total_loss(model, composition_np, content_weight):
#     '''
#     Return the total loss present within the combination image.
#     Total loss is calculated by considering the content, style and variation
#     loss.
#     '''
#     loss = K.variable(0.)

#     layers = dict([(layer.name, layer.output) for layer in model.layers])
#     layer_features = layers['block2_conv2']
#     content_features = layer_features[0, :, :, :]
#     composition_features = layer_features[2, :, :, :]

#     # content loss
#     loss += content_weight * content_loss(content_features,
#                                           composition_features)

#     # style loss calculation, retrieve all layers available for manipulation
#     # within VGG model
#     feature_layers = ['block1_conv2', 'block2_conv2',
#                       'block3_conv3', 'block4_conv3',
#                       'block5_conv3']

#     for layer_name in feature_layers:
#         layer_features = layers[layer_name]
#         style_features = layer_features[1, :, :, :]
#         combination_features = layer_features[2, :, :, :]
#         sl = style_loss(style_features, combination_features)
#         loss += (style_weight / len(feature_layers)) * sl

#     # total variation loss
#     loss += total_variation_weight * total_variation_loss(composition_np)
#     return loss  # return total loss


# def minimize_loss(save_name, loss, grads, composition_np, iterations):
#     '''
#     Using stochastic gradient descent via fmin_l_bfgs_b algorithm. Minimize
#     and balance the loss experienced over the course of 10 iterations.
#     Save the intermediate image combinations.
#     '''
#     x = np.random.uniform(0, 255, (1, height, width, 3)) - 128
#     evaluator = Evaluator(loss, grads, composition_np)

#     print("\n\nProcessing: " + save_name)
#     for i in range(iterations):

#         # print diagnostic information
#         print('Start of iteration', i)
#         start_time = time.time()
#         x, min_val, info = scipy.optimize.fmin_l_bfgs_b(evaluator.loss,
#                                                         x.flatten(),
#                                                         fprime=evaluator.grads,
#                                                         maxfun=20)
#         print('Current loss value:', min_val)
#         end_time = time.time()
#         print('Iteration %d completed in %ds' % (i, end_time - start_time))
#     return x


# def eval_loss_and_grads(x, loss, grads, composition_np):
#     '''
#     Define gradients of the total loss relative to the combination image, in
#     order to minimize the loss.
#     '''
#     x = x.reshape((1, height, width, 3))
#     outputs = [loss]
#     outputs += grads
#     f_outputs = K.function([composition_np], outputs)

#     outs = f_outputs([x])
#     loss_value = outs[0]
#     grad_values = outs[1].flatten().astype('float64')
#     return loss_value, grad_values

# class Evaluator(object):
#     '''
#     Computes the loss and gradient present within the combination image. Phrased
#     as a class to conveniently package retrieval methods for interfacing with 
#     scypy.optimize library.
#     '''

#     def __init__(self, il, ig, comb_np):
#         self.loss_value = None
#         self.grads_values = None
#         self.initial_loss = il
#         self.initial_grad = ig
#         self.comb_np = comb_np

#     def loss(self, x):
#         '''
#         Retrieve the loss value for the combination image.
#         '''
#         assert self.loss_value is None
#         loss_value, grad_values = eval_loss_and_grads(
#             x, self.initial_loss, self.initial_grad, self.comb_np)
#         self.loss_value = loss_value
#         self.grad_values = grad_values
#         return self.loss_value

#     def grads(self, x):
#         '''
#         Retrieve the gradients associated with a particular combination
#         image.
#         '''
#         assert self.loss_value is not None
#         grad_values = np.copy(self.grad_values)
#         self.loss_value = None
#         self.grad_values = None
#         return grad_values

# def import_image(file_name):
#     image = Image.open(file_name)# 1.0 retrieve all input image references
#     # resize (downsample)
#     image = image.resize((height, width), Image.ANTIALIAS)
#     image = np.asarray(image, dtype='float32')  # cast to np array
#     image = np.expand_dims(image, axis=0)  # add placeholder dimension

#     image[:, :, :, 0] -= 103.939  # subtract mean RGB values from ImageNet
#     image[:, :, :, 1] -= 116.779
#     image[:, :, :, 2] -= 123.68
#     # flip RGB pixel ordering => match Simonyan/Sizzerman paper
#     image = image[:, :, :, ::-1]
#     return image


# def export_image(img_array, save_dir):
#     img_array = img_array.reshape((height, width, 3))  # reshape
#     img_array = img_array[:, :, ::-1]  # flip RGB pixel ordering
#     img_array[:, :, 0] += 103.939  # add mean RGB values from ImageNet
#     img_array[:, :, 1] += 116.779
#     img_array[:, :, 2] += 123.68
#     img_array = np.clip(img_array, 0, 255).astype(
#         'uint8')  # trim wayward pixels
#     Image.fromarray(img_array).save(save_dir + '.jpeg', "jpeg")  # save


# def retrieve_inputs():
#     content_names = list(os.path.splitext(
#         each)[0] for each in listdir("../input/contents"))
#     content_paths = glob.glob('../input/contents/*')
#     style_names = list(os.path.splitext(each)[0]
#                        for each in listdir("../input/styles"))
#     style_paths = glob.glob('../input/styles/*')
#     return content_names, content_paths, style_names, style_paths


# def create_composition_single(style_np, content_np, save_name, save_dir):

#     # 3.2 create placeholder image, used to store merger image
#     composition_np = K.placeholder((1, height, width, 3))

#     # 3.3 concatenate the image arrays
#     input_tensor = K.concatenate([content_np,
#                                   style_np,
#                                   composition_np], axis=0)

#     # 4.0 load model, iteratively merge and consolidate the two images
#     # 4.1 load the model
#     model = VGG16(input_tensor=input_tensor,
#                   weights='imagenet', include_top=False)

#     # 4.2 calculate combination loss
#     loss = total_loss(model, composition_np, content_weight)

#     # 4.3 calulate gradients of generated image
#     grads = K.gradients(loss, composition_np)

#     # 4.4 run optimization using previously calculated loss values
#     composition = minimize_loss(save_name, loss, grads, composition_np,1)

#     # 5.0 convert and finalize np array
#     export_image(composition, save_dir)

def main():
    
    #parameterize composition model
    content_weight = 0.025
    style_weight = 5.0
    total_variation_weight = 1.0
    height = 49 #minimum dimensions of 48, VGG16 requirements
    width = 49

    #input images
    content_names, content_paths, style_names, style_paths = retrieve_inputs()

    #for every content image
    for cn, cp in zip(content_names, content_paths):
        mkdir('../output/' + cn) #
        content_np = import_image(cp)

        # create all possible content/style composition images
        for sn, sp in zip(style_names, style_paths):
            style_np = import_image(sp)
            save_name = cn + sn
            save_dir = '../output/' + cn + '/' + sn
            create_composition_single(style_np, content_np, save_name, save_dir)
            
main()



Processing: croat_boatsand_macro
Start of iteration 0
Current loss value: 8.56258e+09
Iteration 0 completed in 30s
